In [1]:
!pip install tensorflow
!pip install gpflow==2.9.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import gpflow
from scipy.stats import norm
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import os
import tensorflow as tf

In [3]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

In [4]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the directory in Google Drive where you want to save the results
save_dir = '/content/drive/MyDrive/HGP_2D_Results'
os.makedirs(save_dir, exist_ok=True)

Mounted at /content/drive


In [5]:
def generate_noisy_data(x, n_points=5, noise_std_range=(0.0, 2.0)):
    X_noisy = []
    y_noisy = []
    noise_levels = []

    for point in x:
        noise_level = np.random.uniform(noise_std_range[0], noise_std_range[1])
        for _ in range(n_points):
            noisy_y = calculate_y(np.array([point])) + np.random.normal(0, noise_level)
            X_noisy.append(point)
            y_noisy.append(noisy_y)
            noise_levels.append(noise_level)

    X_noisy = np.array(X_noisy)
    y_noisy = np.array(y_noisy).reshape(-1, 1)
    noise_levels = np.array(noise_levels).reshape(-1, 1)

    return X_noisy, y_noisy, noise_levels


In [6]:
# assign GP var
gp_var = 1.5

# Function to calculate Y based on the three-hump camel function (Y)
def calculate_y(X):
    X1, X2 = X[:, 0], X[:, 1]  # Extract X1 and X2 from the input array
    term1 = 2 * X1 ** 2
    term2 = -1.05 * X1 ** 4
    term3 = X1 ** 6 / 6
    term4 = X1 * X2
    term5 = X2 ** 2
    return term1 + term2 + term3 + term4 + term5

# Function to calculate Y based on the three-hump camel function (Y)
def calculate_y_true(X):
    X1, X2 = X[:, 0], X[:, 1]  # Extract X1 and X2 from the input array
    term1 = 2 * X1 ** 2
    term2 = -1.05 * X1 ** 4
    term3 = X1 ** 6 / 6
    term4 = X1 * X2
    term5 = X2 ** 2
    return term1 + term2 + term3 + term4 + term5

In [ ]:
# DEFINE HETEROSKEDASTIC GP MODEL + TRAINING
def train_noise_model(X, y):
    kernel = gpflow.kernels.Matern32(lengthscales=0.1)
    noise_model = gpflow.models.GPR(data=(X, y), kernel=kernel, mean_function=None)
    gpflow.utilities.set_trainable(noise_model.likelihood.variance, False)
    opt = gpflow.optimizers.Scipy()
    opt.minimize(noise_model.training_loss, noise_model.trainable_variables)
    return noise_model

def train_heteroskedastic_gp_model(X, y, noise_variance):
    kernel = gpflow.kernels.SquaredExponential(lengthscales=0.3)
    heteroskedastic_gp_model = gpflow.models.GPR(data=(X, y), kernel=kernel, mean_function=None)
    gpflow.utilities.set_trainable(noise_model.likelihood.variance, False)
    # Use the mean noise variance instead of per-data-point variance
    heteroskedastic_gp_model.likelihood.variance.assign(1e-5)
    opt = gpflow.optimizers.Scipy()
    opt.minimize(heteroskedastic_gp_model.training_loss, heteroskedastic_gp_model.trainable_variables)
    return heteroskedastic_gp_model

# Expected Improvement function for minimization
def expected_improvement(y_best, f_mean, f_var):
    variance = np.maximum(f_var, 1e-9)  # Ensure variance is non-negative
    std_dev = np.sqrt(variance)
    delta = y_best - f_mean
    with np.errstate(divide='ignore'):
        Z = delta / std_dev
        ei = delta * norm.cdf(Z) + std_dev * norm.pdf(Z)  # Correctly adjusted for minimization
        ei[std_dev == 0.0] = 0.0
    return ei

In [ ]:
# MAIN SCRIPT
x1_limits = [-2.0, 2.0]
x2_limits = [-2.0, 2.0]

# Define the levels for x1 and x2
x1_manual = np.array([-1.5, 0.5, 1.0])
x2_manual = np.array([-1.5, 0.5, 1.0])

# Initialize an empty DataFrame to store results
results_df = pd.DataFrame(columns=['iteration', 'x1', 'x2', 'y', 'ei', 'LCB', 'y_best', 'true_var', 'pred_var'])

num_iterations = 10  # Define the number of iterations

# Generate combinations of x1 and x2 using itertools.product
x_manual_combinations = np.array(list(itertools.product(x1_manual, x2_manual)))

# Initialize with initial noisy data
X, y, noise_levels = generate_noisy_data(x_manual_combinations, n_points=5)

# Combine all plots into one figure per iteration
for iteration in range(num_iterations + 1):  # Start at iteration 0
    # Train noise model to estimate the noise variance
    noise_model = train_noise_model(X, y)
    pred_noise_var, _ = noise_model.predict_f(X)
    pred_noise_var = pred_noise_var.numpy()

    # Train heteroskedastic GP model
    heteroskedastic_gp_model = train_heteroskedastic_gp_model(X, y, pred_noise_var)

    # Generate grid for prediction
    X1_plot = np.linspace(x1_limits[0], x1_limits[1], 100)
    X2_plot = np.linspace(x2_limits[0], x2_limits[1], 100)
    X1_grid, X2_grid = np.meshgrid(X1_plot, X2_plot)
    X_plot = np.array(list(itertools.product(X1_plot, X2_plot)))

    # Predict f_mean and f_var for the model
    f_mean, f_var = heteroskedastic_gp_model.predict_f(X_plot)
    f_mean = f_mean.numpy().reshape(X1_grid.shape)
    f_var = f_var.numpy().reshape(X1_grid.shape)

    # Calculate true y values for comparison
    y_true = calculate_y_true(X_plot).reshape(X1_grid.shape)

    # Calculate LCB and UCB for plotting
    LCB = f_mean - 1.96 * np.sqrt(f_var)
    UCB = f_mean + 1.96 * np.sqrt(f_var)

    # y_best = minimum value of y observed so far
    y_best = np.min(y)

    # Calculate EI
    ei = expected_improvement(y_best, f_mean.flatten(), f_var.flatten()).reshape(X1_grid.shape)

    if iteration > 0:  # Skip point selection and new data generation for initial iteration
        # Find the point that maximizes EI
        max_ei_index = np.unravel_index(np.argmax(ei), ei.shape)
        x_max_ei = X_plot[max_ei_index]
        y_max_ei_predicted = f_mean[max_ei_index]

        # Find the point with the lowest LCB
        min_lcb_index = np.unravel_index(np.argmin(LCB), LCB.shape)
        x_min_lcb = X_plot[min_lcb_index]
        y_min_lcb_predicted = f_mean[min_lcb_index]

        # Generate new noisy data points for the new X values (EI and LCB points)
        X_new = np.array([x_max_ei, x_min_lcb])
        X_new_noisy, y_new_noisy, _ = generate_noisy_data(X_new, n_points=5)

        # Update the training data with the new noisy points
        X = np.vstack((X, X_new_noisy))
        y = np.vstack((y, y_new_noisy))

        # Store the results in the DataFrame
        new_rows = pd.DataFrame([
            {
                'iteration': iteration,
                'x1': x_max_ei[0],
                'x2': x_max_ei[1],
                'y': y_new_noisy[0, 0],
                'ei': ei[max_ei_index],
                'LCB': None,
                'y_best': y_best,
                'true_var': np.var(y_true.flatten() - f_mean.flatten()),
                'pred_var': f_var[max_ei_index]
            },
            {
                'iteration': iteration,
                'x1': x_min_lcb[0],
                'x2': x_min_lcb[1],
                'y': y_new_noisy[1, 0],
                'ei': ei[min_lcb_index],
                'LCB': LCB[min_lcb_index],
                'y_best': y_best,
                'true_var': np.var(y_true.flatten() - f_mean.flatten()),
                'pred_var': f_var[min_lcb_index]
            }
        ])
    else:  # For iteration 0, store initial points
        new_rows = pd.DataFrame([
            {
                'iteration': iteration,
                'x1': X[i, 0],
                'x2': X[i, 1],
                'y': y[i, 0],
                'ei': None,  # No EI calculation for initial points
                'LCB': None,
                'y_best': y_best,
                'true_var': np.var(y_true.flatten() - f_mean.flatten()),
                'pred_var': None  # No predicted variance for initial points
            } for i in range(X.shape[0])
        ])

    # Drop any all-NA columns before concatenation to avoid FutureWarning
    new_rows = new_rows.dropna(axis=1, how='all')

    # Concatenate with the results DataFrame
    results_df = pd.concat([results_df, new_rows], ignore_index=True)

    # Plotting
    fontsize = 10
    linewidth = 1.5

    # Create a figure with 3 subplots
    fig, axs = plt.subplots(1, 3, figsize=(18, 5), dpi=800)
    fig.tight_layout(pad=3.0)

    # Plot LCB as a contour plot
    contour_1 = axs[0].contourf(X1_grid, X2_grid, LCB, levels=50, cmap='coolwarm', alpha=0.8)
    axs[0].scatter(X[:, 0], X[:, 1], c='white', edgecolor='black', s=20, label='Observed Points')
    axs[0].set_xlabel("x1", fontsize=fontsize)
    axs[0].set_ylabel("x2", fontsize=fontsize)
    axs[0].set_title("Lower Confidence Bound (LCB)", fontsize=fontsize)
    fig.colorbar(contour_1, ax=axs[0])
    axs[0].grid(True, which='major', color='gray', linestyle='--')

    # Plot Mean as a contour plot
    contour_2 = axs[1].contourf(X1_grid, X2_grid, f_mean, levels=50, cmap='coolwarm', alpha=0.8)
    axs[1].scatter(X[:, 0], X[:, 1], c='white', edgecolor='black', s=20, label='Observed Points')
    axs[1].set_xlabel("x1", fontsize=fontsize)
    axs[1].set_ylabel("x2", fontsize=fontsize)
    axs[1].set_title("Predicted Mean", fontsize=fontsize)
    fig.colorbar(contour_2, ax=axs[1])
    axs[1].grid(True, which='major', color='gray', linestyle='--')

    # Plot EI as a contour plot
    contour_3 = axs[2].contourf(X1_grid, X2_grid, ei, levels=50, cmap='coolwarm', alpha=0.8)
    axs[2].scatter(X[:, 0], X[:, 1], c='white', edgecolor='black', s=20, label='Observed Points')
    axs[2].set_xlabel("x1", fontsize=fontsize)
    axs[2].set_ylabel("x2", fontsize=fontsize)
    axs[2].set_title("Expected Improvement (EI)", fontsize=fontsize)
    fig.colorbar(contour_3, ax=axs[2])
    axs[2].grid(True, which='major', color='gray', linestyle='--')

    # Save the combined plot to a file
    plt.savefig(f'HGP_3HC_iteration_{iteration}_2D_combined_plots.png')
    plt.close()

    print(f"Combined figure saved for iteration {iteration}")

# After all iterations have completed, save the final results to a CSV file
results_df.to_csv('2_HeteroNoise_Final_HGP_2D_HGP_Results.csv', index=False)
print("Final results saved to 2D_HGP_Results.csv")
